In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


***
단어 품사 구분하기
***
'I work at google', 'I google at work'의 단어 품사를 구분하는 RNN 코드를 만든다.  
각 단어는 원-핫 인코딩으로 표현한다.  
I   -> [1, 0, 0, 0]  # 대명사  
work  -> [0, 1, 0, 0] # 동사  
at  -> [0, 0, 1, 0] # 전치사  
google -> [0, 0, 0, 1] #명사  

I work at google -> [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1] ]  
I google at work -> [ [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0] ]  

In [2]:
inputs = np.array([ [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1] ],
                    [ [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0] ] 
                  ])
print(inputs.shape)
inputs

(2, 4, 4)


array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]],

       [[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0]]])

In [3]:
tf.set_random_seed(35)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)
print('*' * 100)

print( '출력값 -> {}'.format(outputs) )
print( '상태값 -> {}'.format(state) )
print('*' * 100)

print( '가중치 갯수와 편향값의 개수' )
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) :
     print(v)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
****************************************************************************************************
출력값 -> Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
상태값 -> Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
****************************************************************************************************
가중치 갯수와 편향값의 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [4]:
variable_name = [vari.name for vari in tf.trainable_variables()]
with tf.Session() as sess:
     sess.run(tf.global_variables_initializer())
     outputs_, state_ = sess.run([outputs, state])
     print('*' * 100)
     print( '출력값\n', outputs_, sep='' )
     print('*' * 100)
     
     '''
     첫번째 문장 각 셀의 첫번째 단어의 출력값; 행은 문장의 1단위 품사
     [[[ 0.37247857  0.27281666 -0.03415733] # 이전 단어가 없기때문에 같은 값이 출력된다.
       [-0.12360744 -0.23688789 -0.07885218]
       [-0.27765325 -0.14212623 -0.44862753]
       [-0.7634894  -0.13153659 -0.1189682 ]] # 첫번째 문장 마지막 출력값
     
     두번째 문장 각 셀의 두번째 단어의 출력값
      [[ 0.37247857  0.27281666 -0.03415733] # 이전 단어가 없기때문에 같은 값이 출력된다.
       [-0.62467164  0.70083857  0.17365289]
       [ 0.10053852  0.2770306   0.05076913]
       [-0.05543748 -0.37285638 -0.04752919]]] # 두번째 문장 마지막 출력값
     '''
     
     # 두 문장의 각 셀의 첫 단어의 출력값이 [ 0.37247857  0.27281666 -0.03415733]로 같다.
     # 첫 단어의 출력값이 같은 이유는 이전 단어가 존재하지 않아 이전 상태값이 존재하지 않기때문이다.
     # 두번째 단어부터는 출력값이 [-0.12360744 -0.23688789 -0.07885218], [-0.62467164  0.70083857  0.17365289]처럼 다르게 나타나게된다.
     # -> 상태값이 같더라도 입력값이 다를 수 있고, 입력값이 같더라도 상태값이 다를 수 있기 때문이다. 
     # 두번째 단어부터 출력값이 다르게 나오는 이유는 이전 상태값이 현재 출력에 영향을 주기때문이다.
     
     print( '최종 상태값\n ',state_ , sep='' )
     print('*'* 100)
     
     '''
     첫번째 문장 마지막 상태값
     [[-0.7634894  -0.13153659 -0.1189682 ] # 첫번째 문장 마지막 상태값은 첫번째 문장 마지막 출력값과 같다.
     
     두번째 문장 마지막 상태값
      [-0.05543748 -0.37285638 -0.04752919]] # 두번째 문장 마지막 상태값은 두번째 문장 마지막 출력값과 같다.
     '''
     # 최종 상태값은 최종 출력값과 같다.
     
     
     
     values = sess.run(variable_name)
     print('가중치 -> ', variable_name[0], '\n', values[0], sep='')
     print('편향값 -> ', variable_name[1], '\n', values[1], sep='')

****************************************************************************************************
출력값
[[[ 0.37247857  0.27281666 -0.03415733]
  [-0.12360744 -0.23688789 -0.07885218]
  [-0.27765325 -0.14212623 -0.44862753]
  [-0.7634894  -0.13153659 -0.1189682 ]]

 [[ 0.37247857  0.27281666 -0.03415733]
  [-0.62467164  0.70083857  0.17365289]
  [ 0.10053852  0.2770306   0.05076913]
  [-0.05543748 -0.37285638 -0.04752919]]]
****************************************************************************************************
최종 상태값
 [[-0.7634894  -0.13153659 -0.1189682 ]
 [-0.05543748 -0.37285638 -0.04752919]]
****************************************************************************************************
가중치 -> rnn/basic_rnn_cell/kernel:0
[[ 0.39129794  0.27990448 -0.03417063]
 [-0.13734835 -0.6881115  -0.19269955]
 [-0.19838947  0.15787274 -0.3503854 ]
 [-0.74573535  0.42230952  0.06174737]
 [-0.07638937  0.737661   -0.0250259 ]
 [ 0.22168279  0.6998607   0.48419607]
 [ 0.55390215